In [31]:
# Important libaries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

Speed up the work by working on the stratified sample for now

In [32]:
# Read original CSV data
#vehicledf = pd.read_csv("W:/DS/Programming/Data/CMUvehicle.csv")

In [33]:
#vehicledf.head()

In [34]:
#strat_sample = vehicledf.groupby('type').apply(lambda x: x.sample(frac=0.05, random_state=42))

In [35]:
#strat_sample.to_csv('stratified_sample_CMUvehicle.csv')

# The work

In [36]:
usesample = False

In [37]:
if usesample:
    vehicledf = pd.read_csv("/kaggle/input/vehicle/stratified_sample_CMUvehicle.csv")
else:
    vehicledf = pd.read_csv("E:/Work/DS/Programming/CMUvehicle.csv")

In [38]:
vehicledf.head()

,id,camera_id,plate_number,type,source_datetime,Year,is_registered,direction,gate_id,gate_name
0,242641894,942625003,1กง2501,motorcycle,3/7/2023 4:18:11 PM,2023,True,o,5,วิศวกรรมศาสตร์
1,242641898,1502,กง7559,car,3/7/2023 4:18:10 PM,2023,False,o,30,แยก อมช
2,242641899,1803,กพพ815,motorcycle,3/7/2023 4:18:10 PM,2023,False,o,34,แยกบริหาร
3,242641900,72,5กณ4065,car,3/7/2023 4:18:08 PM,2023,False,o,4,เกษตรศาสตร์
4,242641902,1602,ขค6235,car,3/7/2023 4:18:10 PM,2023,True,o,33,วงเวียนหอนาฬิกา


In [39]:
vehicledf[vehicledf['Year'] != 2023]

,id,camera_id,plate_number,type,source_datetime,Year,is_registered,direction,gate_id,gate_name


Check if camera_id is the same as gate_id by checking for one-to-one relationship:

In [40]:
def isonetoone(df,col1,col2):
    # A short code to check for a one-to-one relationship
    # group rows by camera_id, then find if numbers of unique values in gate_id is one for all groups
    # True means the two columns have a one-to-one relationship
    return df.groupby(col1)[col2].apply(lambda x: x.nunique() == 1).all()

isonetoone(vehicledf,'camera_id','gate_id')

True

The two features have a one-to-one relationship, proceed to drop one of them. In this case, camera_id is drop since is less interpretable. Next, check for one-to-one relationship between gate_id and gate_name:

In [41]:
isonetoone(vehicledf,'gate_id','gate_name')

True

This mean that one can use gate_id to represent both camera_id and gate_name. However, I build a dictionary between gate_id and gate name for further analysis.

In [42]:
GATENAME = pd.Series(vehicledf['gate_name'].values,index=vehicledf['gate_id']).to_dict()

In [43]:
GATENAME

{5: 'วิศวกรรมศาสตร์',
 30: 'แยก อมช',
 34: 'แยกบริหาร',
 4: 'เกษตรศาสตร์',
 33: 'วงเวียนหอนาฬิกา',
 1: 'หน้ามหาวิทยาลัย',
 27: 'วงเวียนสนามเทนนิส',
 25: 'วงเวียนมนุษย์',
 32: 'แยกโรงอาหารใหม่',
 41: 'สวนดอกพาร์ค',
 23: 'วงเวียน SCB',
 24: 'วงเวียนอ่างตาดชมพู',
 36: 'วิศวกรรมศาสตร์ใหม่',
 3: 'คันคลองชลประทาน',
 22: 'แยกอ่างแก้ว',
 26: 'แยกประตูไผ่ล้อม',
 2: 'ไผ่ล้อม',
 29: 'ลานจอดรถ อมช',
 31: 'ลานจอดรถฝายหิน',
 42: 'POC-ENG-in',
 20: 'แยกตึกอธิการบดี',
 18: 'ศึกษาศาสตร์',
 38: 'ลานจอดรถ  S1',
 19: 'ปตท.ใหม่',
 35: 'ลานจอดรถไร่ฟอร์ด',
 28: 'ลานจอดรถหอ 40 ปี',
 21: 'ลานจอดรถหน้ามอ',
 40: 'จอดรถโรงอาหารกลาง',
 37: 'อาคารสำนักงาน 3',
 39: 'ข่วงพยอม'}

Drop unnecessary columns for modelling, including:
* id 
* Unnamed: 1 - resultant column of stratified sampling meaning the previous index 
* type.1 - duplication of type)
* Year - Only instances from 2023 is found
* camera_id - has one-to-one relationship to gate_id
* gate_name - has one-to-one relationship to gate_id

In [44]:
if usesample:
    vehicledf.drop(columns = ['id','Unnamed: 1','type.1','Year','camera_id','gate_name'], inplace = True)
else:
    vehicledf.drop(columns = ['id','Year','camera_id','gate_name'], inplace = True)
vehicledf.head()

,plate_number,type,source_datetime,is_registered,direction,gate_id
0,1กง2501,motorcycle,3/7/2023 4:18:11 PM,True,o,5
1,กง7559,car,3/7/2023 4:18:10 PM,False,o,30
2,กพพ815,motorcycle,3/7/2023 4:18:10 PM,False,o,34
3,5กณ4065,car,3/7/2023 4:18:08 PM,False,o,4
4,ขค6235,car,3/7/2023 4:18:10 PM,True,o,33


In [45]:
vehicledf.shape

(34154261, 6)

Keep only instances that are near to CMU entrances:

In [46]:
'''{19: 'ปตท.ใหม่',
 4: 'เกษตรศาสตร์',
 1: 'หน้ามหาวิทยาลัย',
 36: 'วิศวกรรมศาสตร์ใหม่',
 5: 'วิศวกรรมศาสตร์',
 2: 'ไผ่ล้อม',
 18: 'ศึกษาศาสตร์',
 42: 'POC-ENG-in'}
'''
#entrance_mask = [19,4,1,36,5,2,18,42]
#vehicledf = vehicledf[vehicledf['gate_id'].isin(entrance_mask )]

"{19: 'ปตท.ใหม่',\n 4: 'เกษตรศาสตร์',\n 1: 'หน้ามหาวิทยาลัย',\n 36: 'วิศวกรรมศาสตร์ใหม่',\n 5: 'วิศวกรรมศาสตร์',\n 2: 'ไผ่ล้อม',\n 18: 'ศึกษาศาสตร์',\n 42: 'POC-ENG-in'}\n"

Convert source_datetime to Pandas datetime. Then, filter out vehicles before 08.30 AM and after 15.10 PM

In [47]:
datetime = pd.to_datetime(vehicledf['source_datetime'])
vehicledf['month'] = datetime.dt.month_name()
vehicledf['day'] = datetime.dt.day
vehicledf['hour'] = datetime.dt.hour
vehicledf['min'] = datetime.dt.minute
vehicledf['sec'] = datetime.dt.second
vehicledf.drop(columns = ['source_datetime'], inplace = True)

In [48]:
vehicledf.shape

(34154261, 10)

In [49]:
vehicledf.to_csv('vehicle.csv')

Split data by month for further processing:

In [50]:
#months = vehicledf['month'].unique()
#for m in months:
    #vehicledf[vehicledf['month'] ==  m].to_csv('vehicle_'+m+'.csv')

In [51]:
#vehicledf[vehicledf['month'] ==  'June']

# Encoding

# Clustering with Kmeans